In [1]:
import pandas as pd

In [ ]:
RecVAE_recon = pd.read_csv('/opt/ml/input/code/output/RecVAE_recon.csv').to_numpy()

In [ ]:
EASE_recon = pd.read_csv('/opt/ml/input/code/output/EASE_recon.csv').to_numpy()

In [ ]:
import copy

RecVAE_recon_norm = copy.deepcopy(RecVAE_recon)

for i in range(len(RecVAE_recon_norm)):
    tmp = [score for score in RecVAE_recon_norm[i] if score != -np.inf and score!=np.inf ]

    tmp_mean = np.mean(tmp, axis=0)
    tmp_std = np.std(tmp, axis=0)

    for j, score in enumerate(RecVAE_recon_norm[i]):
        if score != -np.inf and score!=np.inf:
            RecVAE_recon_norm[i][j] = (RecVAE_recon_norm[i][j] - tmp_mean)/tmp_std

In [ ]:
EASE_recon_norm = copy.deepcopy(EASE_recon)

for i in range(len(EASE_recon_norm)):
    tmp = [score for score in EASE_recon_norm[i] if score != -np.inf and score!=np.inf ]

    tmp_mean = np.mean(tmp, axis=0)
    tmp_std = np.std(tmp, axis=0)

    for j, score in enumerate(EASE_recon_norm[i]):
        if score != -np.inf and score!=np.inf:
            EASE_recon_norm[i][j] = (EASE_recon_norm[i][j] - tmp_mean)/tmp_std

In [ ]:
raw_data = pd.read_csv('/opt/ml/input/data/train/train_ratings.csv')

unique_sid = pd.unique(raw_data['item'])
unique_uid = pd.unique(raw_data['user'])
show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))

id2show = list(show2id.keys())

In [90]:
import bottleneck as bn
import numpy as np

submission_norm_item = list()
submission_norm_user = list()

ensemble_recon_norm = np.zeros(shape=RecVAE_recon_norm.shape)

for i in range(len(ensemble_recon_norm)):
    # id2show 과정
    for j in range(len(ensemble_recon_norm[j])):
        if ensemble_recon_norm[i][j]==-np.inf or RecVAE_recon_norm[i][j]==-np.inf:
            ensemble_recon_norm[i][j] = -np.inf
        elif ensemble_recon_norm[i][j]>0 and  RecVAE_recon_norm[i][j]>0 :
            ensemble_recon_norm[i][j] = ensemble_recon_norm[i][j]* RecVAE_recon_norm[i][j]
        else:
            ensemble_recon_norm[i][j] = 0

    idxes = bn.argpartition(-EASE_recon[i], 10)[:10] # 유저에게 추천할 10개 영화를 가져옴

    tmp = list()
    for j in idxes:                    
        tmp.append(list(show2id.keys())[j]) # id2show # = tmp.append(raw_data['item'].unique()[j])
    submission_norm_item.append(tmp)

In [75]:
submission_norm_item = np.array(submission_norm_item).reshape(-1, 1)
submission_norm_user = raw_data['user'].unique().repeat(10)
submission_norm_user = np.array(submission_norm_user).reshape(-1, 1)

submission_norm_result = np.hstack((submission_norm_user,submission_norm_item))

submission_norm_result = pd.DataFrame(submission_norm_result, columns=['user','item'])  
submission_norm_result.to_csv('/opt/ml/input/code/output/submission.csv', index=False)